### Related equations
(arxiv: https://arxiv.org/abs/cond-mat/0207529)

The distribution of real numbers $k$ (between $-Q$ and $Q$) and $\Lambda$ (between $-B$ and $B$) are derived in this script, where $0 < Q \leq \pi$ and $0 < B \leq \infty$. The distributions are normalized by
\begin{equation}
\int_{-Q}^{Q} \rho(k)dk = N/N_a \hspace{0.3cm} (\text{filling}) , \hspace{0.5cm} \int_{-B}^{B} \sigma(\Lambda)d\Lambda = M/N_a \hspace{0.3cm} (\text{density of down spin})
\end{equation}

The equations to solve $\rho(k)$ and $\sigma(\Lambda)$ are
\begin{equation}
\rho(k) = \frac{1}{2\pi}+\cos k\int_{-B}^{B} K(\sin k - \Lambda)\sigma(\Lambda)d\Lambda
\end{equation}
\begin{equation}
\sigma(\Lambda) = \int_{-Q}^{Q} K(\sin k -\Lambda)\rho(k)dk - \int_{-B}^{B} K^2(\Lambda-\Lambda')\sigma(\Lambda')d\Lambda'
\end{equation}
where
\begin{equation}
K(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{8U}{U^2 + 16(\Lambda-\Lambda')^2}\right]
\end{equation}
\begin{equation}
K^2(\Lambda - \Lambda') = \frac{1}{2\pi}\left[\frac{4U}{U^2 + 4(\Lambda-\Lambda')^2}\right]
\end{equation}
Half-filling and $S_z = 0$
$$Q = \pi, \hspace{0.5cm} B = \infty$$ 

Energy is
$$E(M, N) = -2N_a \int_{-Q}^{Q} \rho(k)\cos kdk$$

### Implementation
Rewrite integral equations of $\rho(k)$ and $\sigma(\Lambda)$ with descrete formula (matrix representation).
\begin{equation}
P_k = C^0_k + C_k * K_{kl}S_l
\end{equation}
\begin{equation}
S_l = K_{kl}P_k - K^2_{ll'}S_l
\end{equation}
where $\rho(k) \rightarrow P_k$, $\sigma(\Lambda) \rightarrow S_l$, $\frac{1}{2\pi} \rightarrow C^0_k$, $\cos k\rightarrow C_k$, $K(\sin k-\Lambda)\rightarrow K_{kl}$, and $K^2(\Lambda-\Lambda')\rightarrow K^2_{ll'}$.

Let  If we concatenate $P_k$ and $S_l$, we get
\begin{equation}
\begin{pmatrix}I & -CK \\ -K & I+K^2\end{pmatrix}
\begin{pmatrix}P \\ S\end{pmatrix} 
= \begin{pmatrix}C^0 \\ 0\end{pmatrix}
\end{equation}

In [2]:
import numpy as np

In [14]:
class BatheAnsatzGS(object):
    def __init__(self,U, filling=0.5, nspind=None, ngrid=20):
        self.U = U
        self.filling = filling
        self.nspind = nspind
        # initialization 
        self.Q = np.pi 
        self.B = 20. 
        self.kgrid = np.zeros(ngrid)
        self.lgrid = np.zeros(ngrid)
        self.rhok = np.zeros(ngrid)
        self.sigl = np.zeros(ngrid)
        
    def K_func(self, k, l):
        U = self.U
        return 8.*U/(2.*np.pi*(U**2 + 16*(np.sin(k)-l)**2))
    def K2_func(self, x, xp):
        U = self.U
        return 4.*U/(2.*np.pi*(U**2 + 4*(x-xp)**2))
    def energy_per_site(self):
        return -2.*np.dot(self.rhok, np.cos(self.kgrid))
    def solve_density(self):
        ngrid = self.ngrid
        Cmat = np.zeros((ngrid*2,)*2)
        Kmat = np.zeros((ngrid,ngrid))
        K2mat = np.zeros((ngrid,ngrid))
        for k in range(ngrid):
            for l in range(ngrid):
                Kmat[k,l] = self.K_func(self.kgrid[k], self.lgrid[l])
                K2mat[k,l] = self.K2_func(self.lgrid[k], self.lgrid[l])
        Cmat[:ngrid, :ngrid] = np.eye(ngrid)
        Cmat[:ngrid, ngrid:] = -np.diag(np.cos(self.kgrid)).dot(Kmat)
        Cmat[ngrid:, :ngrid] = -Kmat
        Cmat[ngrid:, ngrid:] = np.eye(ngrid)+K2mat
        
        const_vec = np.zeros(ngrid*2)
        const_vec[:ngrid] = np.ones(ngrid)/(2.*np.pi)
        # TODO solve the matrix*vec = vec equation